In [1]:
import pandas as pd
import numpy as np
import re

In [50]:
df = pd.read_csv('sucide.csv')
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1785 entries, 0 to 1786
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Tweet    1785 non-null   object
 1   Suicide  1785 non-null   object
dtypes: object(2)
memory usage: 41.8+ KB


In [3]:
re.sub(r"[,.?!']", "", df['Tweet'][5])

'@ValenValdez Oh thats good to hear But is it over already Or youll continue it after the Holy week'

In [51]:
def cleaning_one(stri):

    stri = stri.split(" ")
    for i in range(len(stri)):
        for j in ['@', '#']:
            if j in stri[i]:
                stri[i] = stri[i].replace(stri[i], "")
    stri = " ".join(stri)
    stri = re.sub(r"[\W+0-9+]", " ", stri)
    stri = re.findall(r"[A-Za-z]+", stri)
    stri = " ".join(stri)
    stri = stri.lower()

    return stri


df['Tweet'].apply(lambda x: cleaning_one(x))

0                                       making some lunch
1                                      you want his money
2       that crap took me forever to put together i m ...
3             hey jer since when did you start twittering
4       trying out quot delicious library quot with mi...
                              ...                        
1782          i have forgotten how much i love my nokia n
1783    starting my day out with a positive attitude t...
1784       hey it s am give a girl some credit for trying
1785    drunken besties stumble into my room and we ru...
1786    quot i friggin love you quot ron burgundy ridi...
Name: Tweet, Length: 1785, dtype: object

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
 
#nltk.download('stopwords')

In [52]:
def nltk_processing(stri):

    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    stri = stri.split(" ")
    stri = [ps.stem(i) for i in stri if i not in stop_words]
    stri = " ".join(stri)

    return stri

df['Tweet'].apply(lambda x: nltk_processing(x))

0                                              make lunch
1                                 @alexia you want money.
2       @dizzyhrvi crap took forev put together. iâm...
3       @jnaylor #kiwitweet hey jer! sinc start twitte...
4       tri &quot;delici librari 2&quot; mix result th...
                              ...                        
1782                      forgotten much love nokia n95-1
1783    start day posit attitude! to great watch great...
1784    @belledame222 hey, 5 am...giv girl credit trying.
1785    2 drunken besti stumbl room run around sober c...
1786    @dancingbonita &quot;i friggin love you!!!&quo...
Name: Tweet, Length: 1785, dtype: object

In [53]:
df['Tweet'] = df['Tweet'].apply(lambda x: cleaning_one(x)).apply(lambda x: nltk_processing(x))

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import RMSprop

In [60]:
def tf_processing(corpus):

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    sequences = tokenizer.texts_to_sequences(corpus)
    total_words = len(tokenizer.word_index) + 1
    maxlen = max([len(x) for x in sequences])
    sequences = pad_sequences(sequences, maxlen=maxlen)

    return sequences, total_words, maxlen, tokenizer

In [10]:
df['Suicide'].drop_duplicates()

0           Not Suicide post
2    Potential Suicide post 
Name: Suicide, dtype: object

In [61]:
X, total_words, maxlen, tokenizer = tf_processing(df['Tweet'])
y = df['Suicide'].replace({'Not Suicide post':0, 'Potential Suicide post ':1})
y = np.array(y)
y.shape[0]

1785

In [24]:
maxlen

115

In [12]:
def split_data(x_data, y_data, train_split):

    splits = int(x_data.shape[0] * train_split)
    x_train = x_data[:splits]
    y_train = y_data[:splits]
    x_test = x_data[splits:]
    y_test = y_data[splits:]

    return x_train, x_test, y_train, y_test

In [13]:
x_train, x_test, y_train, y_test = split_data(X, y, 0.8)
x_train.shape, y_train.shape

((1428, 115), (1428,))

In [14]:
def create_model(total_words, maxlen):

    model = Sequential([
        Embedding(total_words, 100, input_length=maxlen),
        Bidirectional(LSTM(128)),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid'),
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(learning_rate=0.0001),
                  metrics=['accuracy'])
    
    return model

In [15]:
model = create_model(total_words, maxlen)
history = model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_test, y_test))

Epoch 1/20
45/45 [==============================] - 20s 296ms/step - loss: 0.6711 - accuracy: 0.6275 - val_loss: 0.6403 - val_accuracy: 0.6443
Epoch 2/20
45/45 [==============================] - 11s 243ms/step - loss: 0.6354 - accuracy: 0.6275 - val_loss: 0.6043 - val_accuracy: 0.6443
Epoch 3/20
45/45 [==============================] - 11s 235ms/step - loss: 0.6137 - accuracy: 0.6345 - val_loss: 0.5738 - val_accuracy: 0.6751
Epoch 4/20
45/45 [==============================] - 11s 238ms/step - loss: 0.5874 - accuracy: 0.6940 - val_loss: 0.5380 - val_accuracy: 0.7619
Epoch 5/20
45/45 [==============================] - 11s 239ms/step - loss: 0.5544 - accuracy: 0.7626 - val_loss: 0.5005 - val_accuracy: 0.7955
Epoch 6/20
45/45 [==============================] - 11s 241ms/step - loss: 0.5229 - accuracy: 0.7703 - val_loss: 0.4717 - val_accuracy: 0.8067
Epoch 7/20
45/45 [==============================] - 11s 238ms/step - loss: 0.4935 - accuracy: 0.7808 - val_loss: 0.4389 - val_accuracy: 0.8347

In [16]:
from pickle import dump, load

dump(model, open('model.pkl', 'wb'))

In [17]:
pred = model.predict(x_test)
pred[pred < 0.5] = 0
pred[pred >= 0.5] = 1

12/12 [==============================] - 3s 57ms/step


In [19]:
with open('model.pkl', 'rb') as file:
    # Load the data from the PKL file
    data = load(file)

result = data.predict(x_test)
result[result < 0.5] = 0
result[result >= 0.5] = 1
result

12/12 [==============================] - 2s 51ms/step


array([[1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],

In [22]:
dump(tokenizer, open('tokenizer.pkl', 'wb'))